In [1]:
"""
attaches genuine functionality onto the datastructures / UI elements defined in ui_*.py
"""
%run __init__.py
%load_ext lab_black

In [2]:
import io
import pandas as pd
from IPython.display import (
    # update_display,
    display,
    Image,
    # JSON,
    Markdown,
    # HTML,
    clear_output,
)
import subprocess
import functools
from shutil import copyfile
import getpass
import importlib.util
from halo import HaloNotebook
import pathlib
import typing
from typing import Optional, List, Dict, Type, Optional
from pydantic.dataclasses import dataclass
from pydantic import BaseModel, validator, Field
from jinja2 import Template

import plotly.io as pio
import plotly.graph_objects as go

# widget stuff
import ipywidgets as widgets

# core mf_modules
from ipyautoui import AutoUi, DisplayFiles, AutoUiConfig
from ipyautoui.autoui import display_template_ui_model
from ipyautoui._utils import display_pydantic_json
from pprint import pprint
import importlib.util
import inspect

# display_template_ui_model()

# from this repo
from ipyrun.utils import make_dir, del_matching
from ipyrun.constants import (
    BUTTON_WIDTH_MIN,
    BUTTON_WIDTH_MEDIUM,
    JOBNO_DEFAULT,
    PATH_RUNAPP_HELP,
    PATH_RUNAPPS_HELP,
    DI_STATUS_MAP,
    load_test_constants
)
test_constants = load_test_constants()

from ipyrun.ui_run import *
from enum import Enum, IntEnum
from ipyrun.ui_run import RunActionsUi, RunUi, RunUiConfig
FNM_CONFIG_FILE = "config-shell_handler.json"

def get_mfuser_initials():
    user = getpass.getuser()
    return user[0] + user[2]

In [3]:
class FiletypeEnum(str, Enum):
    input = "in"
    output = "out"
    wip = "wip"

In [4]:
class PyObj(BaseModel):
    path: pathlib.Path
    obj_name: str
    module_name: str = None
    
    @validator("module_name", always=True)
    def _module_name(cls, v, values):
        if v is None:
            return values["path"].stem
        else:
            return v
        
class DisplayfileDefinition(PyObj):
    ftype: FiletypeEnum = None
    ext: str
    
def _get_PyObj(obj: PyObj):
    spec = importlib.util.spec_from_file_location(obj.module_name, obj.path)
    foo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(foo)
    return getattr(foo, obj.obj_name)

def create_pydantic_json_file(pyobj: PyObj, path: pathlib.Path):
    obj = _get_PyObj(pyobj)
    assert str(type(obj)) == "<class 'pydantic.main.ModelMetaclass'>", "the python object must be a pydantic model"
    if not hasattr(obj, "file"):
        from ipyautoui._utils import file
        setattr(obj, 'file', file) 
    assert hasattr(obj, "file"), "the pydantic BaseModel must be extended to have method 'file' for writing model to json"
    myobj = obj()
    myobj.file(path)
    return path
        
def create_displayfile_renderer(ddf: DisplayfileDefinition, fn_onsave: typing.Callable = lambda: None):
    model = _get_PyObj(ddf)
    config_ui = AutoUiConfig(ext=ddf.ext, pydantic_model=model)
    return AutoUi.create_displayfile_renderer(config_autoui=config_ui, fn_onsave=fn_onsave)

class ConfigActionsShell(BaseModel):
    index: int = 0
    key: str = None
    in_batch: bool = True 
    status: str = None
    fpth_script: pathlib.Path
    process_name: str = None
    pretty_name: str = None
    update_config_at_runtime: bool = Field(default=False, description='updates config before running shell command. useful if for example outputs filepaths defined within the input filepaths')
    fdir_appdata: pathlib.Path = Field(default=None, description='working dir for process execution. defaults to script folder if folder not given.')
    displayfile_definitions: typing.List[DisplayfileDefinition] = Field(default=None, description='autoui definitions for displaying files. see ipyautui')
    displayfile_inputs_kwargs: typing.Dict = Field(default_factory=lambda:{})
    displayfile_outputs_kwargs: typing.Dict = Field(default_factory=lambda:{})
    fpths_inputs: List[pathlib.Path] = Field(default = None)
    fpths_outputs: List[pathlib.Path] = Field(default_factory = list)
    fpth_config: pathlib.Path = Field(FNM_CONFIG_FILE, description=f"there is a single unique folder and config file for each RunApp. the config filename is fixed as {FNM_CONFIG_FILE}")
    fpth_runhistory: pathlib.Path = "runhistory.csv"
    fpth_log: pathlib.Path = "log.csv"
    call: str = "python -O"
    params: typing.Dict = {}
    shell_template: str = """\
{{ call }} {{ fpth_script }}\
{% for f in fpths_inputs %} {{f}}{% endfor %}\
{% for f in fpths_outputs %} {{f}}{% endfor %}\
{% for k,v in params.items()%} --{{k}} {{v}}{% endfor %}
"""
    shell: str = ""
    
class DefaultConfigActionsShell(ConfigActionsShell):
    
    @validator("process_name", always=True)
    def _process_name(cls, v, values):
        if v is None:
            return values["fpth_script"].stem.replace('script_','')
        else:
            return v
                       
    @validator("pretty_name", always=True)
    def _pretty_name(cls, v, values):
        if v is None:
            return str(values["process_name"])
        else:
            return v

    @validator("fdir_appdata", always=True)
    def _fdir_appdata(cls, v, values):
        if v is None:
            v=values["fpth_script"].parent
        return v
    
    @validator("fpths_inputs", always=True)
    def _fpths_inputs(cls, v, values):
        if v is None:
            v=[]
            ddfs = [v_ for v_ in values['displayfile_definitions'] if v_.ftype.value == 'in']
            paths = [pathlib.Path('in-'+values['process_name']+ddf.ext) for ddf in ddfs] #+str(values['index']).zfill(2)+'-'
            
            for ddf, path in zip(ddfs, paths):
                if not path.is_file():
                    create_pydantic_json_file(ddf, path)
            v = paths               
        assert type(v)==list, 'type(v)!=list'
        return v
    
    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        if v is None:
            v=[]
        return v 
    
    @validator("key", always=True)
    def _key(cls, v, values):
        if v is None:
            return str(values['index']).zfill(2)
        else:
            return v
        
    @validator("fpth_config", always=True)
    def _fpth_config(cls, v, values):
        return values["fdir_appdata"] / v

    @validator("fpth_runhistory", always=True)
    def _fpth_runhistory(cls, v, values):
        return values["fdir_appdata"] / v

    @validator("shell", always=True)
    def _shell(cls, v, values):
        #pprint(values)
        return Template(values["shell_template"]).render(**values)
    
# template run action callables
def get_status(fpths_inputs, fpths_outputs):
    #['no_outputs', 'up_to_date', 'outputs_need_updating']
    if len(fpths_inputs) ==0: 
        return 'error'
    for f in fpths_outputs:
        if f.is_file() is False:
            return 'no_outputs'
    in_max = max([f.lstat().st_mtime for f in fpths_inputs])
    out_max = max([f.lstat().st_mtime for f in fpths_outputs])
    if in_max > out_max:
        return 'outputs_need_updating'
    else: 
        return 'up_to_date'
    
def update_cls_status(cls=None):
    cls._update_status()
    
def check(config: ConfigActionsShell):
    config.in_batch = True

def uncheck(config: ConfigActionsShell):
    config.in_batch = False
    
def show_files(fpths, class_displayfiles=DisplayFiles, kwargs_displayfiles={}):
    return class_displayfiles([f for f in fpths], **kwargs_displayfiles)
    
def run_shell(shell: str, cls=None): #
    """
    cmd: str, cls=None
    """
    shell = shell.split(" ")
    pr = """  
    """.join(shell)
    display(Markdown(f"{pr}"))
    spinner = HaloNotebook(animation="marquee", text="Running", spinner="dots")
    try:
        spinner.start()
        save = sys.stdout
        sys.stdout = io.StringIO()
        proc = subprocess.Popen(shell)
        proc.wait()
        in_stdout = sys.stdout.getvalue()
        sys.stdout = save
        display(in_stdout)
        spinner.succeed("Finished")
    except subprocess.CalledProcessError as e:
        spinner.fail("Error with Process")
    cls._update_status()
    
def help_config_show(cls=None):
    return display_pydantic_json(cls.config)

def fn_buildrunactions(config: ConfigActionsShell, cls=None) -> RunActions:  # fn_onsave
    # create custom Displayfile 
    user_file_renderers = {}
    for d in config.displayfile_definitions: 
        user_file_renderers.update(create_displayfile_renderer(d, fn_onsave=cls._update_status))    
    cls_display = functools.partial(DisplayFiles, user_file_renderers=user_file_renderers)
    
    run_actions = RunActions(check=functools.partial(check, config),
                             uncheck=functools.partial(uncheck, config),
                             get_status=cls._update_status,
                             help_run_show=functools.partial(show_files, 
                                                           [config.fpth_script],
                                                           class_displayfiles=cls_display,
                                                           kwargs_displayfiles={'auto_open':True}),
                             help_config_show=help_config_show,
                             inputs_show=functools.partial(show_files, 
                                                           config.fpths_inputs,
                                                           class_displayfiles=cls_display,
                                                           kwargs_displayfiles=config.displayfile_inputs_kwargs),
                             outputs_show=functools.partial(show_files, 
                                                           config.fpths_outputs,
                                                           class_displayfiles=cls_display,
                                                           kwargs_displayfiles=config.displayfile_outputs_kwargs),
                             run=functools.partial(run_shell,config.shell)
                            )
    return run_actions

In [5]:


class RunApp:
    def __init__(self,
                 config: typing.Type[BaseModel],
                 cls_ui: typing.Type[widgets.Box] = RunUi,
                 fn_buildactions: typing.Callable[[typing.Type[BaseModel]], RunActions]=fn_buildrunactions
                ):
        """
        The goal of RunApp is to simplify the process of making a functional UI that interacts
        with remote data for use in a Jupyter Notebook or Voila App. 
        
        Args:
            config: typing.Type[BaseModel]
            cls_ui
            fn_buildactions
        """
        self.fn_buildactions = fn_buildrunactions
        self.ui = RunActionsUi() # init with defaults
        self.ui_box = cls_ui(ui_actions=self.ui)
        self.config = config # the setter updates the ui.actions using fn_buildactions. can be updated on the fly
        self._update_status()
        
    @property
    def config(self):
        return self._config
    
    @config.setter
    def config(self, value):
        actions = self.fn_buildactions(value, cls=self)
        #self.actions = self._init_actions(actions)
        self.ui.actions = self._init_actions(actions)
        self._config = value
        
    def _update_status(self):
        st = get_status(self.config.fpths_inputs, self.config.fpths_outputs)
        self.ui.status = st
        self.config.status = st
    
    def _init_run_action(self, action):
        if action is not None:
            try:
                if "cls" in inspect.getfullargspec(action).args:
                    return functools.partial(action, cls=self)
                else:
                    return action
            except:
                print("error inspecting the following:")
                print(action)
                print(type(action))
                print("cls" in inspect.getfullargspec(action).args)
                action()
        else:
            return action

    def _init_actions(
        self, actions: typing.Type[RunActions]
    ) -> typing.Type[RunActions]:
        """this allows us to pass the RunApp object to the Run Actions. TODO: describe better! """
        return type(actions)(
            **{k: self._init_run_action(v) for k, v in actions.dict().items()}
        )
    
    def display(self):
        display(self.ui_box)

    def _ipython_display_(self):
        self.display()
        


In [6]:
PATH_EXAMPLE_SCRIPT = list(test_constants.DIR_EXAMPLE_PROCESS.glob('script*'))[0]

class MyConfigActionsShell(DefaultConfigActionsShell):
    @validator("fpths_outputs", always=True)
    def _fpths_outputs(cls, v, values):
        fdir = values['fdir_appdata']
        nm = values['process_name']
        paths = [fdir / ('output-'+nm+'.csv'), fdir / ('out-' + nm + '.plotly.json')]
        return paths
    
MyConfigActionsShell = functools.partial(MyConfigActionsShell, fpth_script=PATH_EXAMPLE_SCRIPT, 
                            displayfile_definitions=[
                                DisplayfileDefinition(
                                    path=PATH_EXAMPLE_SCRIPT.parent / 'schemas.py',
                                    obj_name='LineGraph',
                                    ext='.lg.json',
                                    ftype=FiletypeEnum.input
                                )])

config = MyConfigActionsShell()
run_app = RunApp(config)
display(run_app)

RunUi(children=(Checkbox(value=False, indent=False, layout=Layout(height='40px', max_width='20px', padding='3p…

PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py')

In [11]:
from typing import Union
class Run(BaseModel):
    fdir_root: pathlib.Path
    process_name: str
    pretty_name: str = None
    fdir_appdata: pathlib.Path = None
    fpth_config: pathlib.Path = None
    
    @validator("fdir_appdata", always=True, pre=True)
    def _fdir_appdata(cls, v, values):
        return values['fdir_root'] / values['process_name']
    
    @validator("fpth_config", always=True, pre=True)
    def _fpth_config(cls, v, values):
        return values['fdir_root'] / FNM_CONFIG_FILE
    
class ConfigBatch(BaseModel):
    fdir_root: pathlib.Path
    #fpth_script: pathlib.Path
    batch_desccription: str = ''
    run_config_model_definition: PyObj = None
    run_config_model: Union[Type[BaseModel],Callable] = Field(None, exclude=True)
    run_app_model_definition: PyObj = None
    run_app_model: Callable = Field(RunApp, exclude=True)
    runs: List[Run] = []
    
    @validator("run_config_model", always=True)
    def _run_config_model(cls, v, values):
        obj_def = values['run_config_model_definition']
        if obj_def is not None and v is None:
            v = _get_PyObj(obj_def)
        return v
    
    @validator("run_app_model", always=True)
    def _run_app_model(cls, v, values):
        obj_def = values['run_app_model_definition']
        if obj_def is not None and v is None:
            v = _get_PyObj(obj_def)
        return v
    
def add(name='name', cls=None):
    fdir_root = cls.config.fdir_root
    Config_ = cls.config.run_config_model
    RunApp_ = cls.config.run_app_model
    run = Run(fdir_root=fdir_root, process_name=name)
    print(run)
    batchconfig = cls.config.copy()
    batchconfig.runs.append(run)
    cls.config = batchconfig
    runconfig = Config_(process_name=run.process_name, pretty_name=run.pretty_name, fdir_appdata=run.fdir_appdata)
    app = RunApp_(runconfig)
    cls.ui_box.runs.add_row(new_key=name, item=app.ui_box)
    
def add_show(cls=None):
    fn_add_show = functools.partial(AddRun, app=cls, fn_add=cls.ui.actions.add, run_name_kwargs={'index': len(cls.config.runs)+1})
    return fn_add_show()

In [12]:
type(MyConfigActionsShell)

functools.partial

In [13]:
config = ConfigBatch(fdir_root=TEST_CONSTANTS.DIR_EXAMPLE_BATCH, run_config_model=MyConfigActionsShell, fpth_script=TEST_CONSTANTS.PATH_EXAMPLE_SCRIPT)
config.run_config_model

<class 'NameError'>: name 'TEST_CONSTANTS' is not defined

In [14]:
add(cls=batch)

<class 'NameError'>: name 'batch' is not defined

In [15]:
AddRun()

In [16]:
def fn_buildbatchactions(config: ConfigBatch, cls=None) -> BatchActions: 
    return BatchActions(add=add, add_show=add_show)
    #pass

In [17]:
class BatchApp():
    def __init__(self,
                 config: typing.Type[BaseModel],
                 cls_ui: typing.Type[widgets.Box] = BatchUi,
                 fn_buildactions: typing.Callable[[typing.Type[BaseModel]], BatchActions]=fn_buildbatchactions
                ):
        """
        The goal of RunApp is to simplify the process of making a functional UI that interacts
        with remote data for use in a Jupyter Notebook or Voila App. 
        
        Args:
            config: typing.Type[BaseModel]
        """
        self.fn_buildactions = fn_buildactions
        self.ui = BatchActionsUi() # init with defaults
        self.ui_box = cls_ui(ui_actions=self.ui)
        self.config = config # the setter updates the ui.actions using fn_buildactions. can be updated on the fly
        #self._update_status()
        
    @property
    def config(self):
        return self._config
    
    @config.setter
    def config(self, value):
        actions = self.fn_buildactions(value, cls=self)
        self.ui.actions = self._init_actions(actions)
        self._config = value
        
    def _update_status(self):
        st = get_status(self.config.fpths_inputs, self.config.fpths_outputs)
        self.ui.status = st
        self.config.status = st
    
    def _init_run_action(self, action):
        if action is not None:
            try:
                if "cls" in inspect.getfullargspec(action).args:
                    return functools.partial(action, cls=self)
                else:
                    return action
            except:
                print("error inspecting the following:")
                print(action)
                print(type(action))
                print("cls" in inspect.getfullargspec(action).args)
                action()
        else:
            return action

    def _init_actions(
        self, actions: typing.Type[BatchActions]
    ) -> typing.Type[BatchActions]:
        """this allows us to pass the RunApp object to the Run Actions. TODO: describe better! """
        return type(actions)(
            **{k: self._init_run_action(v) for k, v in actions.dict().items()}
        )
    
    def display(self):
        display(self.ui_box)

    def _ipython_display_(self):
        self.display()
        
TEST_CONSTANTS = load_test_constants()

config = ConfigBatch(fdir_root=TEST_CONSTANTS.DIR_EXAMPLE_BATCH, run_config_model=MyConfigActionsShell, fpth_script=TEST_CONSTANTS.PATH_EXAMPLE_SCRIPT)
batch = BatchApp(config)
batch

BatchUi(children=(HTML(value='<h1>markdown batch title</h1>'), HBox(children=(Checkbox(value=False, indent=Fal…

In [18]:
batch.config

ConfigBatch(fdir_root=PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph_batch'), batch_desccription='', run_config_model_definition=None, run_app_model_definition=None, runs=[Run(fdir_root=PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph_batch'), process_name='01-None-description', pretty_name=None, fdir_appdata=PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph_batch/01-None-description'), fpth_config=PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph_batch/config-shell_handler.json'))])

In [73]:
batch.ui_box.runs.items

{}

In [69]:
batch.ui.actions.add('01-lean-description')

fdir_root=PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph_batch') process_name='01-lean-description' pretty_name=None fdir_appdata=PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph_batch/01-lean-description') fpth_config=PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph_batch/config-shell_handler.json')


<class 'traitlets.traitlets.TraitError'>: The 'children' trait of a HBox instance contains an Instance of a TypedTuple which expected a Widget, not the RunApp at '0x7f33e1e269d0'.

In [54]:
?TEST_CONSTANTS

Type:        module
String form: <module 'constants' from '/mnt/c/engDev/git_mf/ipyrun/tests/constants.py'>
File:        /mnt/c/engDev/git_mf/ipyrun/tests/constants.py
Docstring:   <no docstring>


In [34]:
?Run

Init signature:
Run(
    *,
    fdir_root: pathlib.Path,
    process_name: str,
    fdir_appdata: pathlib.Path,
    fpth_config: pathlib.Path,
) -> None
Init docstring:
Create a new model by parsing and validating input data from keyword arguments.

Raises ValidationError if the input data cannot be parsed to form a valid model.
File:           ~/miniconda3/envs/ipyautoui/lib/python3.9/site-packages/pydantic/main.cpython-39-x86_64-linux-gnu.so
Type:           ModelMetaclass
Subclasses:     


In [35]:
FiletypeEnum.input.value

'in'

In [14]:
from ipyrun.ui_add import AddRun
from ipyrun.ui_remove import RemoveRun
?AddRun

Init signature:
AddRun(
    app: Type[ipyrun.ui_add.RunApps] = None,
    fn_add: Callable = <function create_runapp at 0x7f55ed53c5e0>,
    run_name_kwargs: Dict = None,
)
Docstring:      <no docstring>
Init docstring:
a ui element for adding new runs to RunApps

Args:
    app (RunApps): the app that will be modified by this class
    fn_add (typing.Callable): a function that is executed to add a new run to the "app"
        on_click of a button. the main app is passed to the function using functools:

Code:
    ```
    def _init_controls(self):
        self.add.on_click(self._add)

    def _add(self, click):
        return functools.partial(self.fn_add, cls=self.app)()
    ```
File:           /mnt/c/engDev/git_mf/ipyrun/src/ipyrun/ui_add.py
Type:           type
Subclasses:     


Init signature:
BatchActions(
    *,
    check: Callable = <function RunActions.<lambda> at 0x7f8d59defe50>,
    uncheck: Callable = <function RunActions.<lambda> at 0x7f8d59defee0>,
    get_status: Callable = <function RunActions.<lambda> at 0x7f8d59deff70>,
    help_ui_show: Callable = <function RunActions.<lambda> at 0x7f8d59dfd040>,
    help_ui_hide: Callable = <function RunActions.<lambda> at 0x7f8d59dfd0d0>,
    help_run_show: Callable = <function RunActions.<lambda> at 0x7f8d59dfd160>,
    help_run_hide: Callable = <function RunActions.<lambda> at 0x7f8d59dfd1f0>,
    help_config_show: Callable = <function RunActions.<lambda> at 0x7f8d59dfd280>,
    help_config_hide: Callable = <function RunActions.<lambda> at 0x7f8d59dfd310>,
    inputs_show: Callable = <function RunActions.<lambda> at 0x7f8d59dfd3a0>,
    inputs_hide: Callable = <function RunActions.<lambda> at 0x7f8d59dfd430>,
    outputs_show: Callable = <function RunActions.<lambda> at 0x7f8d59dfd4c0>,
    outputs_hide: Cal

In [40]:
from ipyrun.ui_run import RunActionsUi, RunUi, RunUiConfig, BatchActions, BatchActionsUi, BatchUi

def _fn_addshow(cls=None, name='name'):
    print(f'_fn_add: name={name}')
    print(f'str(cls)={str(cls)}')
    ui_actions = RunActionsUi()
    keys = cls.apps_box.iterable_keys
    if len(keys) == 0:
        key = 0
    else:
        nums = [int(s.split('-')[0]) for s in keys]
        index = max(nums)
        
    def add_run(cls, name='name'):
        cls.apps_box.add_row(new_key=name)
    #display()    
    return AddRun(app=cls, fn_add=add_run)
    

b_actions = BatchActions(
                         #help_ui_show=None, 
                         #help_ui_hide=None,
                         help_config_show=None,
                         help_config_hide=None,
                         help_run_show=None,
                         help_run_hide=None,
                         inputs_show=None,
                         inputs_hide=None,
                         wizard_show=None,
                         runlog_show=None,
                         run_hide=None, 
                         add_show=_fn_addshow
            )

ui_actions = BatchActionsUi(b_actions)
ui_batch = BatchUi(ui_actions=ui_actions)
ui_batch

BatchUi(children=(HTML(value='<h1>markdown batch title</h1>'), HBox(children=(Checkbox(value=False, indent=Fal…

In [ ]:
def fn_buildbatchactions(config: ConfigActionsShell, cls=None) -> RunActions:  # fn_onsave
    # create custom Displayfile 
    user_file_renderers = {}
    for d in config.displayfile_definitions: 
        user_file_renderers.update(create_displayfile_renderer(d, fn_onsave=fn_onsave))    
    cls_display = functools.partial(DisplayFiles, user_file_renderers=user_file_renderers)
    
    run_actions = BatchActions(check=functools.partial(check, config),
                             uncheck=functools.partial(uncheck, config),
                             get_status=cls._update_status,
                             help_run_show=functools.partial(show_files, 
                                                           [config.fpth_script],
                                                           class_displayfiles=cls_display,
                                                           kwargs_displayfiles={'auto_open':True}),
                             help_config_show=help_config_show,
                             inputs_show=functools.partial(show_files, 
                                                           config.fpths_inputs,
                                                           class_displayfiles=cls_display,
                                                           kwargs_displayfiles=config.displayfile_inputs_kwargs),
                             outputs_show=functools.partial(show_files, 
                                                           config.fpths_outputs,
                                                           class_displayfiles=cls_display,
                                                           kwargs_displayfiles=config.displayfile_outputs_kwargs),
                             run=functools.partial(run_shell,config.shell)
                            )
    return run_actions

In [22]:
AddRun()

In [16]:
max([])

<class 'ValueError'>: max() arg is an empty sequence

In [45]:
from ipyautoui.custom import Dictionary, Array
#run_app.ui.run_form

In [47]:
items = {'test': run_app.ui_box}
d = Dictionary(items, watch_value=False, toggle=False, add_remove_controls='append_only', show_hash=None)
d#.iterable[0].item

Dictionary(children=(HBox(layout=Layout(display='flex', flex='flex-grow')), VBox(children=(HBox(children=(HBox…

In [51]:
run_app1 = RunApp(config)
d.add_row(new_key='test1', item=run_app1.ui_box)

In [15]:
items =[run_app.ui.run_form]
d = Array(items, watch_value=False)
d

<class 'AttributeError'>: 'RunActionsUi' object has no attribute 'run_form'

In [13]:
if __name__ == "__main__":
    display(
        widgets.HBox([
            widgets.HBox(
                [
                    widgets.Accordion(
                        [widgets.Button()], layout=widgets.Layout(width='100%'))]
            ,layout=widgets.Layout(width='100%'))
        ])
    )     

In [8]:
DisplayFiles(config.fpths_outputs)

In [5]:
from ipyrun.utils import write_yaml
import yaml
import json
#write_yaml(config.json())

with open('data.yaml', 'w') as outfile:
    yaml.dump(json.loads(config.json()), outfile, default_flow_style=False)

In [17]:
widgets.Button(tooltip='afds', disabled=True)

Button(disabled=True, style=ButtonStyle(), tooltip='afds')

In [58]:
from pprint import pprint
s = "python -O /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json"
pprint(s)

('python -O '
 '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py '
 '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json '
 '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv '
 '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json')


In [16]:

class RunId(BaseModel):
    """run identifier

    Args:
        BaseModel (pydantic.BaseModel): inherits pydantic

    Returns:
        RunId: RunId object
    """

    index: int = 0
    process_name: str = "process_name"
    pretty_name: str = None
    check: bool = True

    @validator("pretty_name", always=True)
    def _pretty_name(cls, v, values):
        if v is None:
            return str(values["index"]) + " - " + values["process_name"]
        else:
            return v


class BatchId(RunId):
    description: str = Field(
        "",
        description="a description of the batch of RunApps. Displayed as a header to the UI",
    )

    @validator("pretty_name", always=True)
    def _pretty_name(cls, v, values):
        if v is None:
            return values["process_name"]
        else:
            return v
    
    
class BaseConfig(BaseModel):
    run_id: RunId = RunId()
    config_ui: RunUiConfig = RunUiConfig()
    config_actions: typing.Any = None  # this one might get overwritten

In [17]:
class BatchAppConfig(BaseModel):
    # actions: BatchActions = BatchActions() # TODO: add this back in once pydantic has updated such that it can be excluded from the json output. and remade by a validator.
    batch_id: BatchId = BatchId()
    config_ui: RunUiConfig = RunUiConfig()
    config_actions: typing.Any = None  # this one might get overwritten

In [18]:
class RunAppConfig(BaseConfig):
    """generic RunApp configurator definition. this will be serialised to JSON and remade
    for storing the Apps state. it will also be used to generate a RunApp when using the
    "add" button.

    Args:
        BaseModel ([type]): [description]
        
    
    """
    
    actions: RunActions = RunActions() # TODO: add this back in once pydantic has updated such that it can be excluded from the json output. and remade by a validator.

In [19]:
if __name__ == "__main__":
    ?AddRunDialogue

    def add_run_dialogue(cls=None):
        display(AddRunDialogue(cls))

    batch_actions = BatchActions(
        inputs_show=None,
        inputs_hide=None,
        outputs_show=None,
        outputs_hide=None,
        add_show=add_run_dialogue,
    )

    display(RunApps(batch_actions=batch_actions))

Object `AddRunDialogue` not found.


<class 'NameError'>: name 'RunApps' is not defined

In [4]:
def default_runapp_config(
    path_script: pathlib.Path,
    index: int,
    input_models: typing.List[typing.Type[BaseModel]],
    process_name: str = None,
    fdir_appdata: pathlib.Path = None,
    class_displayfiles: typing.Type[DisplayFiles] = DisplayFiles,
) -> typing.Tuple[RunAppConfigShell, RunActions]:

    if fdir_appdata is None:
        fdir_appdata = path_script.parent

    if process_name is None:
        process_name = path_script.stem.replace("script_", "")

    run_id = RunId(process_name=process_name, check=True, index=index)
    paths_inputs = [
        create_inputs_file(model, index, fdir_appdata, process_name=process_name)
        for model in input_models
    ]

    config_actions = ShellHandler(
        fpth_script=path_script,
        fpths_inputs=paths_inputs,
        fpths_outputs=[
            fdir_appdata / f"outputs-{run_id.process_name}-{run_id.index}.csv",
            fdir_appdata / f"outputs-{run_id.process_name}-{run_id.index}.plotly.json",
        ],
        # params={'k':'v'}
    )
    config_runapp = RunAppConfigShell(run_id=run_id, config_actions=config_actions)
    run_actions = RunActions(
        help_config_show=None,
        # help_ui_show=None,
        help_run_show=(
            lambda: display(class_displayfiles([config_actions.fpth_script]))
        ),
        inputs_show=(
            lambda: display(
                class_displayfiles(
                    [f for f in config_actions.fpths_inputs], auto_open=True
                )
            )
        ),
        outputs_show=(
            lambda: display(
                class_displayfiles(
                    [f for f in config_actions.fpths_outputs], auto_open=True
                )
            )
        ),
        run=functools.partial(execute, config_actions.cmd),
    )
    return config_runapp, run_actions

<class 'NameError'>: name 'RunAppConfigShell' is not defined

In [8]:
config.dict()

<class 'NameError'>: name 'config' is not defined

In [10]:
fn_buildoutputs(**config.dict())

<class 'NameError'>: name 'fn_buildoutputs' is not defined

In [ ]:
def fn_updateoutputs(**kwargs):
    fdir = kwargs['fdir_appdata']
    nm_in = kwargs['fpths_inputs'][0].with_suffix('').stem
    outputs = [
        fdir / (nm_in.replace('input', 'output') + '.csv'),
        fdir / (nm_in.replace('input', 'output') + '.plotly.json')
    ]
    return outputs

In [12]:
# /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv
# C:\engDev\git_mf\ipyrun\tests\examples\line_graph\output-line_graph.plotly.json

In [13]:
#config.dict()

In [14]:
#fn_updateoutputs(config).dict()

In [15]:
def fn_updateoutputs(config):
    kwargs = config.dict()
    fdir = kwargs['fdir_appdata']
    nm_in = kwargs['fpths_inputs'][0].with_suffix('').stem
    outputs = [
        fdir / (nm_in.replace('input', 'output') + '.plotly.json'),
        fdir / (nm_in.replace('input', 'output') + '.csv')
    ]
    kwargs['fpths_outputs'] = outputs
    #pprint(kwargs)
    newconfig = type(config)(**kwargs)
    return newconfig

<class 'AttributeError'>: 'RunActions' object has no attribute 'runlog_show'

In [277]:
run_app.__dict__.keys()

dict_keys(['config', 'fn_buildactions', 'ui'])

In [283]:
from ipyautoui.constants import BUTTON_HEIGHT_MIN
widgets.Checkbox(icon='fa-add')

Checkbox(value=False)

In [282]:
flag = widgets.Button(layout={'width':'10px','height':'20px'}, button_style='danger')
check_w = widgets.Checkbox(indent=False)
widgets.HBox([flag, run_app.ui.run_form])

In [235]:
subprocess.run("python -O /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv", shell=True)

CompletedProcess(args='python -O /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv', returncode=1)

In [243]:
s = "python -O /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json"
subprocess.run(s.split(" "))

CompletedProcess(args=['python', '-O', '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py', '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json', '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv', '/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json'], returncode=1)

In [195]:
run_app.config.dict()

{'in_batch': True,
 'fpth_script': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py'),
 'process_name': 'process_name',
 'pretty_name': 'process_name',
 'fdir_appdata': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph'),
 'displayfile_definitions': [{'path': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/schemas.py'),
   'obj_name': 'LineGraph',
   'module_name': 'schemas',
   'ext': '.lg.json'}],
 'displayfile_inputs_kwargs': {},
 'displayfile_outputs_kwargs': {},
 'patterns_inputs': ['input-*'],
 'fpths_inputs': [PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json'),
  PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json')],
 'fpths_outputs': [PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json'),
  PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.csv')],
 'fpth_co

In [108]:
subprocess.run("python -O /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json", shell=True)

CompletedProcess(args='python -O /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py /mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json', returncode=1)

In [105]:
run_app.ui.actions.run()

```python
-O
/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py
/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/input-line_graph.lg.json```

Output()

''

<class 'AttributeError'>: 'NoneType' object has no attribute 'config'

In [73]:
config.fpths_inputs

[]

In [71]:
config.fpths_outputs[1]

PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/output-line_graph.plotly.json')

In [9]:
def create_ShellAppConfig(fpth_script, fdir_appdata=None, run_index=0, process_name=None, pretty_name=None, check=True):
    
    if process_name is None:
        process_name = f"{fpth_script.stem}-{run_index}"

    return ShellAppConfig(run_id=RunId(index=run_index, process_name=process_name, pretty_name=pretty_name, check=check), 
                          run_actions=
                         )

In [48]:
obj = PyObj(path='/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/schemas.py',obj_name='LineGraph')
_get_PyObj(obj)

schemas.LineGraph

In [6]:
import re

re.match('[A-Z]{2,6}-[1-9][0-9]*', 'AHU-1023234234')

<re.Match object; span=(0, 14), match='AHU-1023234234'>

In [42]:
spec

ModuleSpec(name='', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7fa0d83b3520>, origin='/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/schemas.py')

In [ ]:
f = lambda:['output-*']
f()

In [62]:
from ipyautoui.test_schema import TestAutoLogic

In [35]:
from ipyautoui.displayfile import DisplayFile

DisplayFile('/mnt/c/engDev/git_mf/ipyrun/tests/constants.py')

In [74]:
list(test_constants.DIR_EXAMPLE_PROCESS.glob('*inputs*'))[0]

PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/inputs-linegraph-line_graph-0.lg.json')

{'call': 'python -O',
 'cmd_template': '{{ call }} {{ fpth_script }}{% for f in fpths_inputs %} '
                 '{{f}}{% endfor %}{% for f in fpths_outputs %} {{f}}{% endfor '
                 '%}{% for k,v in params.items()%} --{{k}} {{v}}{% endfor %}\n',
 'fdir_appdata': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph'),
 'fpth_config': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/config-shell_handler.json'),
 'fpth_runhistory': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/runhistory.csv'),
 'fpth_script': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py'),
 'fpths_outputs': [],
 'params': {},
 'patterns_inputs': ['input-*'],
 'patterns_outputs': ['output-*']}


<class 'KeyError'>: 'fpths_inputs'

In [ ]:
class RunAppConfigShell(RunAppConfig):
    config_actions: ShellHandler = None

    @validator("config_actions", always=True)
    def _config_actions(cls, v, values):
        run_id = values["config_actions"] 
        config_actions = values["config_actions"] 

ShellHandler(
        fpth_script=path_script,
        fdir_appdata=path_script.parent,
        fpths_inputs=paths_inputs,
        fpths_outputs=[
            fdir_appdata / f"outputs-{run_id.process_name}-{run_id.index}.csv",
            fdir_appdata / f"outputs-{run_id.process_name}-{run_id.index}.plotly.json",
        ],
        # params={'k':'v'}
    )
        #return values["fdir_appdata"] / v


if __name__ == "__main__":
    display(
        Markdown(
            """
### RunAppConfigShell

Extend the `RunAppConfig`
    """
        )
    )
    display(Markdown("`>>> display(RunAppConfigShell())`"))
    config_runapp = RunAppConfigShell()
    display(RunAppConfigShell().dict())

In [9]:
path =pathlib.Path('.')
list(path.glob('inputs-*'))

[]

In [11]:
if __name__ == "__main__":
    from ipyrun.constants import load_test_constants
    from ipyautoui.autoui import AutoUi, AutoUiConfig
    from ipyautoui.displayfile import DisplayFiles, DisplayFile

    test_constants = load_test_constants()
    sys.path.append(str(test_constants.DIR_EXAMPLE_PROCESS))
    from schemas import LineGraph
    import functools
    import sys

    config_autoui = AutoUiConfig(pydantic_model=LineGraph, ext=".lg.json")
    LineGraphUi = AutoUi.create_displayfile(config_autoui)

    def line_graph_prev(path):
        display(LineGraphUi(path))

    user_file_renderers = {".lg.json": line_graph_prev}
    DisplayFiles = functools.partial(
        DisplayFiles, user_file_renderers=user_file_renderers
    )  # overwrite the DisplayFiles class with the .lg.json file renderer baked in
    test_constants = load_test_constants()

    PATH_SCRIPT = list(test_constants.DIR_EXAMPLE_PROCESS.glob(pattern="script*"))[0]
    tu = default_runapp_config(
        PATH_SCRIPT, 1, input_models=[LineGraph], class_displayfiles=DisplayFiles
    )
    run = RunApp.from_config(*tu)
    display(run)

<class 'NameError'>: name 'default_runapp_config' is not defined

In [14]:
tu[0].dict()

{'run_id': {'index': 1,
  'process_name': 'line_graph',
  'pretty_name': '1 - line_graph',
  'check': True},
 'config_ui': {'include_show_hide': True},
 'config_actions': {'call': 'python -O',
  'fpth_script': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/script_line_graph.py'),
  'fdir_appdata': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph'),
  'fpths_inputs': [PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/inputs-linegraph-line_graph-1.lg.json')],
  'fpths_outputs': [PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/outputs-line_graph-1.csv'),
   PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/outputs-line_graph-1.plotly.json')],
  'params': {},
  'fpth_config': PosixPath('/mnt/c/engDev/git_mf/ipyrun/tests/examples/line_graph/config-shell_handler.json'),
  'cmd_template': '{{ call }} {{ fpth_script }}{% for f in fpths_inputs %} {{f}}{% endfor %}{% for f in fpths_outputs %} {{f}}{% endfor %}{% for

In [ ]:
from io import StringIO


def create_runapp_linegraph(
    path_script=PATH_SCRIPT,
    input_models=[LineGraph],
    class_displayfiles=DisplayFiles,
    cls=None,
):
    if cls is not None:
        index = len(cls.apps)
    else:
        index = 0
    return RunApp.from_config(
        *default_runapp_config(
            path_script,
            index,
            input_models,
            class_displayfiles=class_displayfiles,
        )
    )


def add_linegraph_dialogue(cls=None):
    display(AddRunDialogue(cls, add_cmd=create_runapp_linegraph))


batch_actions = BatchActions(
    inputs_show=None,
    inputs_hide=None,
    outputs_show=None,
    outputs_hide=None,
    add_show=add_linegraph_dialogue,
)
batch = RunApps(batch_actions=batch_actions, apps=[create_runapp_linegraph()])
display(batch)
# create_runapp_linegraph(2)